In [15]:
import os
import numpy as np
import re
import pickle
import sys

import geopandas as gpd
import pandas as pd
from shapely.geometry import Point

from ProcessEventsFunctions import *
# from Convert_to_Profiles_Functions import *
# from Get_Events_Functions import *

sys.path.insert(1, 'Old')
from Steef_Functions import *

home_dir = '/nfs/a319/gy17m2a/PhD/'
home_dir2 = '/nfs/a161/gy17m2a/PhD/'

In [22]:
with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_future_new.pickle", 'rb') as handle:
    events_props_dict_future = pickle.load(handle)
    
# # with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_profiles_dict_future.pickle", 'rb') as handle:
# #     event_profiles_dict_future = pickle.load(handle)    
    
# with open(home_dir +  f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/events_dict_future_new.pickle", 'rb') as handle:
#     events_dict_future = pickle.load(handle)        

In [24]:
events_props_dict_future_trim = [d for d in events_props_dict_future if d.get('em') != 'bb192']

In [4]:
tbo_vals = pd.read_csv(home_dir + 'datadir/RainGauge/interarrival_thresholds_CDD_noMissing.txt')
# tbo_vals = tbo_vals[tbo_vals['Lon']!=-999.0]
# Check if the points are within the areas
tbo_vals = check_for_gauge_in_areas(tbo_vals, home_dir, ['NW', 'NE', 'ME', 'SE', 'SW'])
tbo_vals.loc[tbo_vals['within_area'] == 'NW, C', 'within_area'] = 'NW'
tbo_vals.loc[tbo_vals['within_area'] == 'ME, SE', 'within_area'] = 'ME'

In [6]:
ems_future = ['bb225']

In [9]:
def process_events_alltogether(home_dir, time_period, ems, tb0_vals, save_dir):
    events_dict = {}
    event_props_ls = events_props_dict_future
    event_profiles_dict = {}
    print(len(event_props_ls))

    for em in ems:
        print(em)
        this_em = []
        
        for gauge_num in range(0, 1294):
            if gauge_num not in [444, 827, 888]:
                if gauge_num % 100 == 0:
                    print(f"Processing gauge {gauge_num}")
                indy_events_fp = home_dir + f"ProcessedData/IndependentEvents/UKCP18_30mins/{time_period}/{em}/{gauge_num}/WholeYear/"

                files = [f for f in os.listdir(indy_events_fp) if f.endswith('.csv')]
                files = np.sort(files)

                for event_num, file in enumerate(files):
                    fp = indy_events_fp + f"{file}"
                    if '2080' in fp:
                        continue
                    try:
                        # Get event
                        this_event = read_event(gauge_num, fp)

                        # Get times and precipitation values
                        event_times = this_event['times']
                        event_precip = this_event['precipitation (mm)']

                        # Apply the function to adjust the dates in the 'times' column
                        event_times_fixed = event_times.apply(adjust_feb_dates)

                        # Create the DataFrame with corrected times
                        event_df = pd.DataFrame({'precipitation (mm)': event_precip, 'times': event_times_fixed})

                        # Create characteristics dictionary
                        event_props = create_event_characteristics_dict(event_df)

                        # Add the duration
                        event_props['dur_for_which_this_is_amax'] = get_dur_for_which_this_is_amax(fp)
                        # Add gauge number and ensemble member
                        event_props['gauge_num'] = gauge_num
                        event_props['area'] = tb0_vals.iloc[gauge_num]['within_area']
                        event_props['em'] = em
                        event_props['filename'] = file

#                         event_profiles = create_profiles_dict(event_df)

#                         events_dict[f"{em}, {gauge_num}, {event_num}"] = event_df
                        event_props_ls.append(event_props)
                        this_em.append(event_props)
#                         event_profiles_dict[f"{em}, {gauge_num}, {event_num}"] = event_profiles
                    except:
                        print(fp)
                        
        print(f"Finished {em}, {len(this_em)}")           
        
#         with open(save_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/events_dict_future_new.pickle", 'wb') as handle:
#             pickle.dump(events_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

#         with open(save_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_profiles_dict_future_new.pickle", 'wb') as handle:
#             pickle.dump(event_profiles_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)

        with open(save_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_future_new.pickle", 'wb') as handle:
            pickle.dump(event_props_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)                       

    return events_dict, event_props_ls, event_profiles_dict

In [24]:
# # Now you can call the function for both time periods
events_dict_future, event_props_dict_future, event_profiles_dict_future = process_events_alltogether(home_dir2, 'Future',ems_future, tbo_vals, home_dir)

# with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/events_dict_future.pickle", 'wb') as handle:
#     pickle.dump(events_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_profiles_dict_future", 'wb') as handle:
#     pickle.dump(event_profiles_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)

# with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_future.pickle", 'wb') as handle:
#     pickle.dump(event_props_dict_future, handle, protocol=pickle.HIGHEST_PROTOCOL)            


In [19]:
with open(home_dir + f"ProcessedData/AMAX_Events/UKCP18_30mins/Future/event_props_dict_future_new.pickle", 'wb') as handle:
    pickle.dump(event_props_ls, handle, protocol=pickle.HIGHEST_PROTOCOL)         

### Check if there are the right number of events for em BB192